In [1]:
%load_ext autoreload
%autoreload 2

Test new physics simulation, cell classes, and mothermachine geometry.

In [1]:
from SyMBac.cell_simulation import run_simulation, create_space, trench_creator, step_and_update3
from SyMBac.trench_geometry import trench_creator2
from SyMBac.mothermachine_geometry import Mothermachine, MothermachinePart
from SyMBac.cell import Cell, CellIDFactory, create_width_callable, create_lysis_callable
import numpy as np
from pymunk.pyglet_util import DrawOptions
import pymunk
import pyglet
import time
gravity = 0 
pix_mic_conv = 0.065
trench_length = 12
trench_width = 1.1
cell_max_length = 4
cell_width = 1
max_length_var = 0
width_var = 0
lysis_p = 0.0 # TODO This bugs. Cells are being thrown out of trenches.
show_window = True
phys_iters = 10
sim_length = 100
save_dir = "/tmp/"

space = create_space()
space.gravity = 0, gravity  # arbitrary units, negative is toward trench pole
space.collision_slop = 0.
dt = 1 / 100  # time-step per frame
scale_factor = 1 / pix_mic_conv # resolution scaling factor

trench_length = trench_length * scale_factor
trench_width = trench_width * scale_factor
channel_width = 2 * trench_length
mothermachine = Mothermachine(
    space=space,
    trench_width=trench_width,
    trench_length=trench_length,
    trench_spacing=4*trench_width,
    num_trenches=4,
    channel_width=channel_width,
)

print(mothermachine)


cell_id_factory = CellIDFactory()
cells = []
for i in mothermachine.get_trench_ids():
    cell_length = cell_max_length * 0.9 * scale_factor
    trench_position = mothermachine.get_trench_position(trench_id=i)
    cell_position_x = trench_position[0] + trench_width * 0.5
    if mothermachine.is_top_trench(trench_id=i):
        cell_position_y = trench_position[1] + trench_length - cell_length * 0.5
        mother_above_daughter = True
    else:
        cell_position_y = trench_position[1] + cell_length * 0.5
        mother_above_daughter = False
    cell_pos = (cell_position_x, cell_position_y)
    cell = Cell(
        length=cell_length,
        width=cell_width * scale_factor,
        position=cell_pos,
        id_factory=cell_id_factory,
        self_id = cell_id_factory.get_next_id(),
        angle=np.pi/2,
        max_length=cell_max_length * scale_factor,
        get_width_daughter=create_width_callable(max_width=0.9*trench_width),
        get_lysis_event=create_lysis_callable(lysis_probability=lysis_p),
        mother_above_daughter=mother_above_daughter,
    )
    cells.append(cell)
    cell.shape.color = (255, 0, 0, 255)
    space.add(cell.body, cell.shape)


if show_window:    
    window = pyglet.window.Window(700, 700, "SyMBac", resizable=True)
    options = DrawOptions()
    options.shape_outline_color = (10,20,30,40)
    bb_mothermachine = mothermachine.get_bounding_box(which=MothermachinePart.MOTHERMACHINE)
    scale_x = window.width / (bb_mothermachine.right - bb_mothermachine.left)
    scale_y = window.height / (bb_mothermachine.top - bb_mothermachine.bottom)
    scale_factor = min(scale_x, scale_y)
    window.view = window.view.scale((scale_factor, scale_factor, 1))

    @window.event
    def on_draw():
        window.clear()
        space.debug_draw(options)

    # key press event
    @window.event
    def on_key_press(symbol, modifier):

        # key "E" get press
        if symbol == pyglet.window.key.E:
            # close the window
            window.close()

sim_progress = [0]
cell_timeseries = []
if show_window:
    pyglet.clock.schedule_interval_for_duration(
        step_and_update3, 
        interval=dt, 
        duration=sim_length, 
        cells=cells, 
        space=space, 
        phys_iters=phys_iters,
        cell_timeseries=cell_timeseries, 
        sim_progress=sim_progress, 
        sim_length=sim_length,
        mothermachine=mothermachine
    )
    pyglet.app.run()

_ = input()
pyglet.app.exit()
window.close()


Trench Width: 16.92, Trench Length: 184.62, Trench Spacing: 67.69, Number of Trenches: 4, Channel Width: 369.23, Segment Thickness: 1.00, Left Margin: 67.69, Right Margin: 67.69, Flow Direction: ChannelFlowDirection.RIGHT


Test with configuration objects.

In [1]:
import pyglet

from SyMBac.simulation import Simulation2
from SyMBac.config import TEST_CONFIG_CELL, TEST_CONFIG_MOTHERMACHINE, TEST_CONFIG_SIMULATION

simulation = Simulation2(
    config_simulation=TEST_CONFIG_SIMULATION,
    config_cell=TEST_CONFIG_CELL,
    config_mothermachine=TEST_CONFIG_MOTHERMACHINE,
)

simulation.run_simulation()

/home/lady5906/workspace_python/symbac/SyMBac/simulation.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
simulation.draw_simulation_OPL()

In [2]:
from SyMBac.simulation import gen_cell_props_for_draw

cell_timeseries_list = [simulation.cells]
ID_props = [cell.get_curvature_properties() for cell in simulation.cells]
gen_cell_props_for_draw(cell_timeseries_lists, ID_props)

Cell has 50 negative pixels in y coordinate, try increasing your offset
Cell has 51 negative pixels in y coordinate, try increasing your offset
Cell has 731 negative pixels in y coordinate, try increasing your offset
Cell has 731 negative pixels in y coordinate, try increasing your offset
Cell has 51 negative pixels in y coordinate, try increasing your offset
Cell has 51 negative pixels in y coordinate, try increasing your offset
Cell has 730 negative pixels in y coordinate, try increasing your offset
Cell has 730 negative pixels in y coordinate, try increasing your offset
Cell has 51 negative pixels in y coordinate, try increasing your offset
Cell has 51 negative pixels in y coordinate, try increasing your offset
Cell has 729 negative pixels in y coordinate, try increasing your offset
Cell has 729 negative pixels in y coordinate, try increasing your offset
Cell has 50 negative pixels in y coordinate, try increasing your offset
Cell has 51 negative pixels in y coordinate, try increasin